In [1]:
from wdtools import *

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
set_ID = 'Set001'

In [3]:
#gdf = combine_merged_data(setID = set_ID)

In [4]:
gdf = gpd.read_file(os.path.join(inpath + f'\\{outfolder}\\', f'matched_records_{set_ID}.shp'), driver='ESRI Shapefile')

In [5]:
# this will take about 17 minutes
all_taxlot = combine_taxlot()

In [20]:
n_gdf = merge_matched(setID = set_ID, all_taxlot = all_taxlot)

it is about 8.38% of data in the original 1420 records unmatched
there are 1 records (0.07% of the original records) without parcel id


C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [25]:
n_gdf['lots'] = n_gdf['lots'].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))

In [28]:
n_gdf.dtypes

wdID            object
trsqq           object
parcel_id       object
loc_desc        object
city            object
county          object
site_name       object
site_desc       object
latitude       float64
longitude      float64
coord_src       object
doc_name        object
doc_link        object
isbatfile       object
status_nm       object
receiveddt      object
responsedt      object
reissuance      object
project_id     float64
site_id          int64
notes           object
lots            object
lot             object
cnt_code         int64
ORTaxlot        object
year             int64
IDyear          object
record_ID        int64
geometry      geometry
dtype: object

In [30]:
n_gdf[['record_ID', 'wdID', 'ORTaxlot', 'geometry']].to_file(os.path.join(inpath + f'\\{outfolder}\\', f'combined_records_in_{set_ID}.shp'), 
                                                  driver='ESRI Shapefile')